In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train, y_train) , (x_test,y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
x_train , x_test = x_train/255.0 , x_test/255.0

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28)),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [7]:
model.compile(optimizer = 'adam',
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [8]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2193 - accuracy: 0.9351
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0962 - accuracy: 0.9707
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0678 - accuracy: 0.9784
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0525 - accuracy: 0.9833
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0420 - accuracy: 0.9865


In [9]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0756 - accuracy: 0.9784


[0.07564523071050644, 0.9783999919891357]

Automate search of hyperparameter using Keras-Tuner

In [10]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.4 MB/s eta 0:00:00


In [11]:
import kerastuner as kt

<ipython-input-11-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [12]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))

  hp_units = hp.Int('units',min_value=16,max_value=512,step=16)
  model.add(keras.layers.Dense(units=hp_units,activation='relu'))
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(10))

  model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

  return model

In [13]:
#Search strategy

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     project_name='intro_to_kt')
#Other flavors are Bayesian Optimization,Random Search,Sklearn

In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience = 5)

tuner.search(x_train,y_train,epochs=50,validation_split=0.2,callbacks=[stop_early])

Trial 30 Complete [00h 00m 38s]
val_accuracy: 0.3215833306312561

Best val_accuracy So Far: 0.6775833368301392
Total elapsed time: 00h 12m 50s


In [18]:
#get the best hyper parameter
best_hp = tuner.get_best_hyperparameters()[0]
print("Best value for unit hyper parameter is:",best_hp.get('units'))

Best value for unit hyper parameter is: 192


In [19]:
model= tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(192,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10,activation='softmax')
    ])

In [20]:
model.compile(optimizer = 'adam',
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [21]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2665 - accuracy: 0.9213
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1215 - accuracy: 0.9638
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0896 - accuracy: 0.9723
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0697 - accuracy: 0.9781
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0596 - accuracy: 0.9815


In [22]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 1ms/step - loss: 0.0752 - accuracy: 0.9768


[0.07521720975637436, 0.9768000245094299]